This is to scrap lyrics from the website.

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

Very old method: get the element and extract the relevant text. Because the website dynamic action making it hard to extract all links.

In [4]:
from bs4 import BeautifulSoup

# Open and read the file
with open('element_all_song.txt', 'r') as f:
    html = f.read()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Get the divs for the song titles and the a tags for the song URLs
titles = soup.select('div.mini_card-title')
urls = soup.select('mini-song-card a')

# Extract the URLs from each a tag
urls = [url['href'] for url in urls]

# Extract the Chinese and English title from each title div
chinese_titles = []
english_titles = []

for title in titles:
    parts = title.text.split(' (')
    chinese_titles.append(parts[0])
    if len(parts) > 1:
        english_titles.append(parts[1].replace(')', ''))
    else:
        english_titles.append(None)

# Print the URLs and titles
for url, chi_title, eng_title in zip(urls, chinese_titles, english_titles):
    print(f"URL: {url}")
    print(f"Chinese Title: {chi_title}")
    print(f"English Title: {eng_title}")
    print()


URL: https://genius.com/Dpr-ian-no-blueberries-lyrics
Chinese Title: No Blueberries
English Title: None

URL: https://genius.com/Dpr-ian-nerves-lyrics
Chinese Title: Nerves
English Title: None

URL: https://genius.com/Dpr-ian-so-beautiful-lyrics
Chinese Title: So Beautiful
English Title: None

URL: https://genius.com/Dpr-live-kiss-me-lyrics
Chinese Title: KISS ME
English Title: None

URL: https://genius.com/Dpr-ian-scaredy-cat-lyrics
Chinese Title: Scaredy Cat
English Title: None

URL: https://genius.com/Dpr-ian-dope-lovers-lyrics
Chinese Title: Dope Lovers
English Title: None

URL: https://genius.com/Dpr-ian-ballroom-extravaganza-lyrics
Chinese Title: Ballroom Extravaganza
English Title: None

URL: https://genius.com/Dpr-ian-welcome-to-the-show-lyrics
Chinese Title: Welcome To The Show
English Title: None

URL: https://genius.com/Dpr-ian-peanut-butter-and-tears-lyrics
Chinese Title: Peanut Butter & Tears
English Title: None

URL: https://genius.com/Dpr-ian-mood-lyrics
Chinese Title: M

Make it a data frame.

In [5]:
df = pd.DataFrame({
    'URL': urls,
    'Title': chinese_titles,
    'English Title': english_titles
})

print(df)

                                                  URL                  Title   
0    https://genius.com/Dpr-ian-no-blueberries-lyrics         No Blueberries  \
1            https://genius.com/Dpr-ian-nerves-lyrics                 Nerves   
2      https://genius.com/Dpr-ian-so-beautiful-lyrics           So Beautiful   
3          https://genius.com/Dpr-live-kiss-me-lyrics                KISS ME   
4       https://genius.com/Dpr-ian-scaredy-cat-lyrics            Scaredy Cat   
5       https://genius.com/Dpr-ian-dope-lovers-lyrics            Dope Lovers   
6   https://genius.com/Dpr-ian-ballroom-extravagan...  Ballroom Extravaganza   
7   https://genius.com/Dpr-ian-welcome-to-the-show...    Welcome To The Show   
8   https://genius.com/Dpr-ian-peanut-butter-and-t...  Peanut Butter & Tears   
9              https://genius.com/Dpr-ian-mood-lyrics                   Mood   
10  https://genius.com/Dpr-live-dpr-ian-and-peace-...  Diamonds + And Pearls   
11            https://genius.com/Dpr-liv

For each song page, extract the lyrics

In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm  # This library is for progress bars. It's optional.

# Initialize an empty list to store the lyrics.
lyrics_list = []

# Loop over each URL in the DataFrame.
for url in tqdm(df['URL']):
    # Send a GET request to the URL.
    response = requests.get(url)
    
    # Parse the HTML content of the page with BeautifulSoup.
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the lyrics div and extract the text.
    lyrics_div = soup.find('div', attrs={"data-lyrics-container": "true"})
    if lyrics_div:  # Check if the div was found
        lyrics = lyrics_div.get_text(separator='\n')
    else:
        print(f"Failed to find lyrics for URL: {url}")
        lyrics = None
    
    # Append the lyrics to the lyrics_list.
    lyrics_list.append(lyrics)

# Add the lyrics_list as a new column in the DataFrame.
df['Lyrics'] = lyrics_list


 83%|████████▎ | 39/47 [00:09<00:01,  5.08it/s]

Failed to find lyrics for URL: https://genius.com/Dpr-ian-the-gods-can-bleed-lyrics


 87%|████████▋ | 41/47 [00:09<00:01,  4.74it/s]

Failed to find lyrics for URL: https://genius.com/Dpr-ian-avalon-interlude-band-version-lyrics


100%|██████████| 47/47 [00:10<00:00,  4.37it/s]


In [12]:
df.to_csv('dprian_lyrics_full.csv', index=False)
df

,URL,Title,English Title,Lyrics
0,https://genius.com/Dpr-ian-no-blueberries-lyrics,No Blueberries,None,[Chorus: DPR IAN & \nCL\n]\nI like no blueberr...
1,https://genius.com/Dpr-ian-nerves-lyrics,Nerves,None,"[Verse 1]\nHey girl, I heard you're leaving to..."
2,https://genius.com/Dpr-ian-so-beautiful-lyrics,So Beautiful,None,[Verse 1]\nI think I've got my sight for you\n...
3,https://genius.com/Dpr-live-kiss-me-lyrics,KISS ME,None,[Verse 1]\nLet's listen to the stars\nWe just ...
4,https://genius.com/Dpr-ian-scaredy-cat-lyrics,Scaredy Cat,None,[Verse 1]\nI think I'm scared of all the signs...
5,https://genius.com/Dpr-ian-dope-lovers-lyrics,Dope Lovers,None,"[Chorus]\nYou're smiling, I’m sorry\nShe's vib..."
6,https://genius.com/Dpr-ian-ballroom-extravagan...,Ballroom Extravaganza,None,[Verse 1]\nThe crowd has come to a halt\nThe b...
7,https://genius.com/Dpr-ian-welcome-to-the-show...,Welcome To The Show,None,[Verse 1]\nJust stay for the show\nNo candle l...
8,https://genius.com/Dpr-ian-peanut-butter-and-t...,Peanut Butter & Tears,None,[Intro]\nSunday morning karaoke\nForgot my bil...
9,https://genius.com/Dpr-ian-mood-lyrics,Mood,None,[Intro]\nMITO\n[Chorus]\nColor in my moods (My...
